#DimUser

In [0]:
df = spark.read.format("parquet")\
          .load("abfss://bronze@spotifystorageaccount01.dfs.core.windows.net/DimUser")

In [0]:
display(df)

### Import Spark Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import os
import sys

project_path = os.path.join(os.getcwd(),'..','..')
sys.path.append(project_path)


In [0]:
import sys
sys.path.append('/Workspace/Users/akhil.ramireddy@outlook.com/spotify')
from Utils.Transformations import reusable

#AutoLoader


In [0]:
df_user = spark.readStream.format("cloudFiles")\
               .option("cloudFiles.format", "parquet")\
               .option("cloudFiles.schemaLocation","abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimUser/checkpoint")\
               .option("schemaEvaluationMode", "addColumns")\
               .load("abfss://bronze@spotifystorageaccount01.dfs.core.windows.net/DimUser")

In [0]:
display(df_user)

In [0]:
df_user = df_user.withColumn("user_name",upper(col("user_name")))
display(df_user)

In [0]:
df_user_obj = reusable()

df_user = df_user_obj.dropColumns(df_user,['_rescued_data'])
df_user = df_user.dropDuplicates(['user_id'])
display(df_user)

In [0]:
df_user.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimUser/checkpoint")\
       .trigger(once=True)\
       .option("path", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimUser/data")\
       .toTable("spotify_catalog.silver.DimUser")
       

#DimArtist

In [0]:
df_artist = spark.readStream.format("cloudFiles")\
               .option("cloudFiles.format", "parquet")\
               .option("cloudFiles.schemaLocation","abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimArtist/checkpoint")\
                .option("schemaEvaluationMode", "addColumns")\
               .load("abfss://bronze@spotifystorageaccount01.dfs.core.windows.net/DimArtist")

In [0]:
display(df_artist)

In [0]:
df_artist = df_artist.withColumn("artist_name", upper(col("artist_name")))

display(df_artist)

In [0]:
df_artist_obj = reusable()

df_artist = df_artist_obj.dropColumns(df_artist, ['_rescued_data'])
df_artist = df_artist.dropDuplicates(['artist_id'])

display(df_artist)

In [0]:
df_artist.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimArtist/checkpoint")\
       .trigger(once=True)\
       .option("path", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimArtist/data")\
       .toTable("spotify_catalog.silver.DimArtist")
       
       

#DimTrack

In [0]:
df_track = spark.readStream.format("cloudFiles")\
               .option("cloudFiles.format", "parquet")\
               .option("cloudFiles.schemaLocation","abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimTrack/checkpoint")\
               .option("schemaEvaluationMode", "addColumns")\
               .load("abfss://bronze@spotifystorageaccount01.dfs.core.windows.net/DimTrack")

In [0]:
display(df_track)


In [0]:
df_track = df_track.withColumn("durationFlag",when(col("duration_sec")<150,"low")\
                                             .when(col("duration_sec")<300,"medium")\
                                             .otherwise("high"))


df_track = df_track.withColumn("track_name", regexp_replace(col("track_name"), "-", " "))

display(df_track)

In [0]:
df_track_obj = reusable()

df_track = df_track_obj.dropColumns(df_track, ['_rescued_data'])
df_track = df_track.dropDuplicates(['artist_id'])

display(df_track)

In [0]:
df_track.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimTrack/checkpoint")\
       .trigger(once=True)\
       .option("path", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimTrack/data")\
       .toTable("spotify_catalog.silver.DimTrack")
       

#DimDate


In [0]:
df_date = spark.readStream.format("cloudFiles")\
               .option("cloudFiles.format", "parquet")\
               .option("cloudFiles.schemaLocation","abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimDate/checkpoint")\
               .option("schemaEvaluationMode", "addColumns")\
               .load("abfss://bronze@spotifystorageaccount01.dfs.core.windows.net/DimDate")

In [0]:
display(df_date)

In [0]:
df_date_obj = reusable()

df_date = df_date_obj.dropColumns(df_date, ['_rescued_data'])

display(df_date)

In [0]:
df_date.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimDate/checkpoint")\
       .trigger(once=True)\
       .option("path", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/DimDate/data")\
       .toTable("spotify_catalog.silver.DimDate")
       

# FactStream

In [0]:
df_fs = spark.readStream.format("cloudFiles")\
               .option("cloudFiles.format", "parquet")\
               .option("cloudFiles.schemaLocation","abfss://silver@spotifystorageaccount01.dfs.core.windows.net/FactStream/checkpoint")\
               .option("schemaEvaluationMode", "addColumns")\
               .load("abfss://bronze@spotifystorageaccount01.dfs.core.windows.net/FactStream")

In [0]:
display(df_fs)

In [0]:
df_fs_obj = reusable()

df_fs = df_fs_obj.dropColumns(df_fs, ['_rescued_data'])

display(df_fs)

In [0]:
df_fs.writeStream.format("delta")\
       .outputMode("append")\
       .option("checkpointLocation", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/FactStream/checkpoint")\
       .trigger(once=True)\
       .option("path", "abfss://silver@spotifystorageaccount01.dfs.core.windows.net/FactStream/data")\
       .toTable("spotify_catalog.silver.FactStream")
       